In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torchvision.transforms import transforms
from PIL import Image
import numpy as np
import os
import torch.nn as nn
from matplotlib.pyplot import imread

Fill in the correct file paths below.

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/New folder/model.39"
TRAINING_IMAGES_DIRECTORY = "/content/drive/MyDrive/sketches_hw-20240703T225724Z-001/sketches_hw"
TESTING_IMAGE_TRUCK = "/content/drive/MyDrive/New folder/Truck.png"
TESTING_IMAGE_TIGER = "/content/drive/MyDrive/New folder/Tiger.png"
TESTING_IMAGE_SWAN = "/content/drive/MyDrive/New folder/Swan.png"
TESTING_IMAGE_MERMAID = "/content/drive/MyDrive/New folder/Mermaid.png"
TESTING_IMAGE_ELEPHANT = "/content/drive/MyDrive/New folder/Elephant.png"

In [ ]:
class Sketch_a_Net_CNN(torch.nn.Module):
    def __init__(self):
        super(Sketch_a_Net_CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=15, stride=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=7, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout2d()
        )

        self.layer7 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout2d()
        )

        self.fc1 = nn.Sequential(
            nn.Linear(512, 5)
        )


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return(out)

In [ ]:
def sketchnet():
    model = Sketch_a_Net_CNN()
    return model

In [ ]:
def load_model():
    model = sketchnet()
    model_path = MODEL_PATH
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint)
    return model

In [ ]:
def construct_transformer():
    """construct transformer for images"""
    transformer = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.Resize((225, 225)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transformer

In [ ]:
def load_categories():
    """load the classification id-name dictionary"""
    classes = os.listdir(TRAINING_IMAGES_DIRECTORY)
    classes.sort()
    class_to_idx = {i: classes[i] for i in range(len(classes))}
    return class_to_idx

Assign Predicted Category to correct value

In [ ]:
def main(image_path):
    # load classification categories
    categories = load_categories()
    print(categories)

    # setup the device for running
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # load model and set to evaluation mode
    model = load_model()
    model.to(device)
    model.eval()

    # set image transformer
    transformer = construct_transformer()
    image = Image.open(image_path)
    image = image.convert('RGB')

    image = transformer(image)
    image = image.view([1, 3, 225, 225])
    image = image.to(device)

    # run the forward process
    prediction = model(image)
    _, cls = torch.max(prediction, dim=1)

    # output
    index = cls.data.tolist()[0]
    print(index)

    # Assign PRED_CATEGORY to correct value
    pred_category = categories[index]
    print("The predicted category is " + pred_category)

Call main() with the appropriate parameters in order to test each of your images.

In [ ]:
main(TESTING_IMAGE_TRUCK)

{0: 'elephant', 1: 'mermaid', 2: 'swan', 3: 'tiger', 4: 'truck'}


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
main(TESTING_IMAGE_TIGER)


{0: 'elephant', 1: 'mermaid', 2: 'swan', 3: 'tiger', 4: 'truck'}
4
The predicted category is truck


In [ ]:
main(TESTING_IMAGE_SWAN)


{0: 'elephant', 1: 'mermaid', 2: 'swan', 3: 'tiger', 4: 'truck'}
1
The predicted category is mermaid


In [ ]:
main(TESTING_IMAGE_MERMAID)


{0: 'elephant', 1: 'mermaid', 2: 'swan', 3: 'tiger', 4: 'truck'}
1
The predicted category is mermaid


In [ ]:
main(TESTING_IMAGE_ELEPHANT)


{0: 'elephant', 1: 'mermaid', 2: 'swan', 3: 'tiger', 4: 'truck'}
3
The predicted category is tiger
